# Продвинутый Python, семинар 2 

**Лектор:** Петров Тимур

**Семинаристы:** Петров Тимур, Коган Александра, Романченко Полина

**Spoiler Alert:** в рамках курса нельзя изучить ни одну из тем от и до досконально (к сожалению, на это требуется больше времени, чем даже 3 часа в неделю). Но мы попробуем рассказать столько, сколько возможно :)

Думая о том, где можно было бы хорошо попрактиковать анализ данных, меня вдруг осенило, что одно из явных приложений - это аналитика игр

Никакая успешная команда, выступающая на чемпионатах (будь то The International в Dota2 или же Worlds в LoL), всегда должна иметь аналитика, который будет анализировать игры и на их основе находить инсайты по тому, какую композицию выбрать или же различные синергии.

Поэтому мы будем анализировать игру, которая очень нравится самому лектору - TFT (TeamFight Tactics). Это автобаттлер, в котором ты размещаешь фигурки, апгейдишь их, кладешь итемы и так далее.

![TFT](https://genapilot.ru/wp-content/uploads/2020/03/teamfight-tactics-vyhodit-na-mobilnyh-ustrojstvah-s-ios-i-android.jpg)

Поэтому попрактикуемся на базовом датасете: [тык](https://www.kaggle.com/datasets/pvillanueva13/tft-35-500-gm-matches?select=tft_3.5_last_500_gm_match_info.csv)

## Грузим данные и библиотеки

In [1]:
!pip install python-dateutil

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import numpy as np
import pandas as pd
import datetime as dt
from dateutil.parser import parse
import dateutil
import pytz

In [3]:
!wget https://github.com/Palladain/Deep_Python/raw/main/Seminars/Seminar_2/archive.zip

--2022-09-17 12:32:42--  https://github.com/Palladain/Deep_Python/raw/main/Seminars/Seminar_2/archive.zip
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Palladain/Deep_Python/main/Seminars/Seminar_2/archive.zip [following]
--2022-09-17 12:32:42--  https://raw.githubusercontent.com/Palladain/Deep_Python/main/Seminars/Seminar_2/archive.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 694969 (679K) [application/zip]
Saving to: ‘archive.zip’

archive.zip         100%[===================>] 678.68K  --.-KB/s    in 0.02s   

2022-09-17 12:32:43 (36.4 MB/s) - ‘archive.zip’ saved [694969/694969]



In [4]:
!unzip archive.zip

Archive:  archive.zip
  inflating: items.json              
  inflating: tft_3.5_last_500_gm_match_info.csv  
  inflating: tft_3.5_last_500_gm_unit_info.csv  
  inflating: traits.json             


## Что у нас есть?

Наши данные - это информация о 500 матчах игроков и фигурках, которые были поставлены.

А также jsonы с информацией про предметы и особенности

### Задание 1.

Загрузите данные и посмотрите на них

In [6]:
matches = pd.read_csv('tft_3.5_last_500_gm_match_info.csv')
matches.sample(5)

,match_id,puuid,challenger,galaxy,placement,game_datetime,Astro,Chrono,DarkStar,Set3_Celestial,...,Starship,Demolitionist,Mercenary,Set3_Blademaster,Paragon,Protector,Set3_Sorcerer,StarGuardian,MechPilot,SpacePirate
580,NA1_3570182972,rJsH0NSNrtKVlFMkuI4DVD6ZbMjLLvFURSxvWYDwepUAP4...,0,Superdense Galaxy,5,2020-09-10 22:14:38,0,1,0,0,...,0,1,1,0,0,0,0,0,0,0
2461,NA1_3573461173,BHi50xOpe1zGRAMWv1foN09g500RTk-bD7s5QKyzTC5cXh...,1,Plunder Planet,6,2020-09-13 11:15:11,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2627,NA1_3573633633,0uh3-P0lR2dDrADgrtlkc5JvXqmbpRu6a3HrBMzIJ9dcYR...,0,Manatee's Delight,4,2020-09-13 17:12:52,0,0,0,1,...,0,0,0,0,1,1,2,2,0,0
3246,NA1_3575090829,foNUZlvWzlPI0VFz-Xe2RD9gOoD9mMFerw-eyHQS1LJHZK...,0,Salvage World,7,2020-09-14 18:45:52,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
725,NA1_3570457985,zm68Kgbx3LwxS_dIjc7DlWhINcebcV3V-fAY-_KENlOwuF...,0,Plunder Planet,6,2020-09-11 03:11:00,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
heroes = pd.read_csv('tft_3.5_last_500_gm_unit_info.csv')
heroes.sample(5)

,match_id,puuid,champion,tier,item_set_1,item_set_2,item_set_3,traits_1,traits_2,traits_3
5175,NA1_3575298121,Kdf451aitI52E4WEMmylY3ImCM712lVw2l2ekcqwacfXDs...,Syndra,2,78.0,44.0,12.0,Star Guardian,Sorcerer,NaN
920,NA1_3576253604,o6ndCy1GPvJrzElbSxk1SC_XrUtiMKZFHzYfBFBgY39x7S...,Jayce,2,59.0,66.0,36.0,Space Pirate,Vanguard,NaN
5135,NA1_3575320119,YZGJxr1voNX3LO1SF5ni5lM7_u1h8gyzP2aWuadU0CsYFl...,Fiora,3,35.0,35.0,NaN,Cybernetic,Blademaster,NaN
1257,NA1_3576165241,4HvGaTbmdKZTGmeoohzdS3pVz28-1WcWM4yBZDkao5sFjJ...,Caitlyn,2,NaN,NaN,NaN,Chrono,Sniper,NaN
6715,NA1_3575071096,L7BhIfvSuA60yFlkMt1Y19OOj2COEBgXIuEV3vLvWTiZdW...,Fiora,2,88.0,NaN,NaN,Cybernetic,Blademaster,NaN


Отлично! Что мы видим:

1. Куча каких-то непонятных id

2. Поля, которые нам мало что говорят (если вы только не заядлый игрок в TFT)

3. И NaN, как же без них

## Задание 2.

Посчитайте число строк в датасете matches и иосмотрите на число уникальных puuid и match_id в датасете с матчами.

Сделайте вывод, сколько было матчей и сколько игроков играло

In [8]:
len(matches) # число строк

4000

In [9]:
len(matches.puuid.unique()) # число уникальных puuid

787

In [10]:
len(matches.match_id.unique()) # число уникальных матчей

500

## Задание 3.

Определить, сколько было челленджеров (это люди, у которых самый высокий ранг, столбец challenger)

In [11]:
len(matches[matches.challenger == 1].puuid.unique()) # уникальные челленджеры
# при сравнении, как и в numpy, выдается маска, по которой можно отфильтровать значения

83

## Задание 4.

Посмотреть, сколько участников было в каждом матче

In [12]:
matches.groupby(['match_id']).agg({"puuid": "nunique"}).sort_values("puuid")
# делаем groupby по каждому матчу и аггрегируем по уникальному числу puuid, затем сортируем, чтобы увидеть, и в начале, и в конце по 8 участников

,puuid
match_id,
NA1_3569517250,8
NA1_3573894123,8
NA1_3573873244,8
NA1_3573857760,8
NA1_3573838008,8
...,...
NA1_3571083877,8
NA1_3571071704,8
NA1_3571058349,8


## Задание 5.

Создайте колонку winner, где 1-4 место - это победа, а 5-8 - проигрыш

А также создайте колонку galaxy_numb, где каждой галактике будет присвоен свой собственный номер

И после этого посмотреть на распределение мест у challengers и распределение галактик в играх

In [13]:
pd.factorize(matches.galaxy)[0] # функция, которая позволяет закодировать строки с помощью номера
# [0] - сама факторизация, [1] - соответствие номера и строки

array([0, 0, 0, ..., 1, 1, 1])

In [14]:
matches['galaxy_numb'] = pd.factorize(matches.galaxy)[0]

In [16]:
matches[matches.galaxy == 'Superdense Galaxy'].sample(5)

,match_id,puuid,challenger,galaxy,placement,game_datetime,Astro,Chrono,DarkStar,Set3_Celestial,...,Demolitionist,Mercenary,Set3_Blademaster,Paragon,Protector,Set3_Sorcerer,StarGuardian,MechPilot,SpacePirate,galaxy_numb
1806,NA1_3572263487,sgQfTsOBiTDGzH51dUC0yjlg13lMdBfDwiQ_Qo-B2w7fQQ...,0,Superdense Galaxy,7,2020-09-12 10:30:25,0,0,0,1,...,0,0,0,0,1,1,1,0,0,6
3196,NA1_3574966529,5Dm0Z8JlXefiM2LLveiHOujFTcF_mWxATFtF989HnUlMjU...,0,Superdense Galaxy,5,2020-09-14 16:00:40,0,0,0,0,...,1,1,0,0,0,1,0,1,0,6
2398,NA1_3573379820,SLYkdFEVSjvFAdyPmfODuusLFLibvit2mtoD3kWKyl4NBL...,0,Superdense Galaxy,7,2020-09-13 08:26:20,0,0,2,2,...,0,0,0,0,1,0,0,0,0,6
1779,NA1_3572250234,Y0emdKI096n9-Rb0L4_I4pP_tmn_kwCtKiEjIsBRrxRqX_...,1,Superdense Galaxy,4,2020-09-12 09:37:45,0,1,0,0,...,0,0,0,0,0,0,0,0,0,6
2656,NA1_3573641768,4HvGaTbmdKZTGmeoohzdS3pVz28-1WcWM4yBZDkao5sFjJ...,0,Superdense Galaxy,1,2020-09-13 18:15:29,0,1,0,0,...,1,1,0,0,0,0,0,0,0,6


In [51]:
matches["winner"] = matches.placement.apply(lambda x: int(x <= 4)) # как сделать с помощью apply

In [18]:
matches["winner"] = (matches.placement <= 4).astype("int") # как сделать с помощью сравнения (превращаем в int)
matches.head(5)

,match_id,puuid,challenger,galaxy,placement,game_datetime,Astro,Chrono,DarkStar,Set3_Celestial,...,Mercenary,Set3_Blademaster,Paragon,Protector,Set3_Sorcerer,StarGuardian,MechPilot,SpacePirate,galaxy_numb,winner
0,NA1_3569517250,7AKHCtexkJJo8p04lg4shtx7n7R-uH6pQwPhBCEofkIJca...,0,Treasure Trove,1,2020-09-10 05:18:02,0,0,3,2,...,0,0,0,1,0,0,0,0,0,1
1,NA1_3569517250,0StQeXmDacsAikvEcY2ubarDZqzlEsaYsZHO6UT-UgBFY7...,0,Treasure Trove,2,2020-09-10 05:18:02,0,0,0,0,...,0,0,1,1,2,2,0,0,0,1
2,NA1_3569517250,Y0emdKI096n9-Rb0L4_I4pP_tmn_kwCtKiEjIsBRrxRqX_...,1,Treasure Trove,3,2020-09-10 05:18:02,0,1,0,0,...,1,0,0,0,0,0,0,0,0,1
3,NA1_3569517250,1vCCOdxLrdYa6NQRhywKKdZ0UNdX0nsa15Zpqo03k6QFs6...,1,Treasure Trove,4,2020-09-10 05:18:02,0,0,1,0,...,0,0,1,1,2,1,0,0,0,1
4,NA1_3569517250,mC0hMfKYhejc1U1aJE7iTOTn6TUDkY4apfQhdw2Pvp_jV3...,0,Treasure Trove,5,2020-09-10 05:18:02,0,0,0,1,...,0,0,1,1,0,0,0,0,0,0


In [19]:
matches[matches.challenger == 1].placement.describe() # посмотреть распределения побед у challengers

count    750.000000
mean       3.582667
std        2.134457
min        1.000000
25%        2.000000
50%        3.000000
75%        5.000000
max        8.000000
Name: placement, dtype: float64

Видим, что игроки с высоким рейтингом очень часто выигрывают! (То есть это не дело случая)

## Datetime

Чтобы решать задачи со временем, можно было бы просто вычленить из строки нужное поле и жить спокойно. Но вот, допустим, что вы хотите посмотреть не только час, но и минута, день, месяц, год и так далее. А еще у вас там часовой пояс есть какой-нибудь.

Выглядит не очень. Вот ровно для такого и существует библиотека datetime! С ее помощью можно достать из строки дату и время, преобразовать ее, привести к общему часовому поясу, посчитать разницу во времени и так далее

[Документация](https://docs.python.org/3/library/datetime.html)

Есть еще полезная надстройка над datetime - dateutil, которая в некоторых моментах позволяет более удобно работать со временем и его преобразованием

[Документация](https://dateutil.readthedocs.io/en/stable/index.html)

Какие функции есть у datetime? Начнем с преобразований из строк

* datetime.fromisoformat(s) - работает, если у вас формат выглядит как

```
YYYY-MM-DD[*HH[:MM[:SS[.fff[fff]]]][+HH:MM[:SS[.ffffff]]]]
YYY-MM-DD - год-месяц-день
HH:MM:SS - часы-минуты-секунды (может быть еще после точки миллисекунды)
+HH::MM:SS - часовой пояс не через название
```

* datetime.strptime(s, format) - по заданному шаблону преобразовать в datetime (однако тут надо задать правильный шаблон)

Форматы:

```
%a  Locale’s abbreviated weekday name.
%A  Locale’s full weekday name.      
%b  Locale’s abbreviated month name.     
%B  Locale’s full month name.
%c  Locale’s appropriate date and time representation.   
%d  Day of the month as a decimal number [01,31].    
%f  Microsecond as a decimal number [0,999999], zero-padded on the left
%H  Hour (24-hour clock) as a decimal number [00,23].    
%I  Hour (12-hour clock) as a decimal number [01,12].    
%j  Day of the year as a decimal number [001,366].   
%m  Month as a decimal number [01,12].   
%M  Minute as a decimal number [00,59].      
%p  Locale’s equivalent of either AM or PM.
%S  Second as a decimal number [00,61].
%U  Week number of the year (Sunday as the first day of the week)
%w  Weekday as a decimal number [0(Sunday),6].   
%W  Week number of the year (Monday as the first day of the week)
%x  Locale’s appropriate date representation.    
%X  Locale’s appropriate time representation.    
%y  Year without century as a decimal number [00,99].    
%Y  Year with century as a decimal number.   
%z  UTC offset in the form +HHMM or -HHMM.
%Z  Time zone name (empty string if the object is naive).    
%%  A literal '%' character.
```

* dateutil.parser.parse(s) - спарсить в datetime (без лишних заморочек с форматом)

In [20]:
d = '2022-04-26T15:14:32.001+03:00'

dt.datetime.fromisoformat(d)

datetime.datetime(2022, 4, 26, 15, 14, 32, 1000, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800)))

In [21]:
from dateutil.parser import parse
#А теперь поменяем: DD-MM-YYYY
d = '05-12-2022 15:14:32.001GMT'
format = '%d-%m-%Y %H:%M:%S.%f%Z'

dt.datetime.strptime(d, format)

datetime.datetime(2022, 12, 5, 15, 14, 32, 1000)

In [23]:
d = '05-14-2022 15:14:32.001GMT'
print(parse(d, dayfirst=True)) #Только здесь надо было указать, что день идет первее, чем месяц
# Обратите внимание, что есть не самые очевидные, например, 22-05-04 (04.05.2022) - такое он не спарсит нормально, нужны strptime

d = '22-06-05'
print(parse(d, dayfirst=True))

tzinfos = {'MSK': 10800}
d = 'Sat Oct 11 17:13:46 MSK 2003'
print(parse(d, tzinfos=tzinfos)) # И даже так он умеет, но надо задавать все нужные вам таймзоны

2022-05-14 15:14:32.001000+00:00
2005-06-22 00:00:00
2003-10-11 17:13:46+03:00


In [26]:
dt.datetime.fromtimestamp(1745603400) #можно вытаскивать также и с timestamp время (секунды от 01.01.1970)

datetime.datetime(2025, 4, 25, 17, 50)

Мы получили инстанс в виде datetime, из которого можно спокойно вытащить примерно все:

In [27]:
parse(d, tzinfos=tzinfos)

datetime.datetime(2003, 10, 11, 17, 13, 46, tzinfo=tzoffset('MSK', 10800))

In [28]:
d = parse(d, tzinfos=tzinfos)
print(d.year, d.month, d.day, d.hour, d.minute, d.second, d.tzinfo)

2003 10 11 17 13 46 tzoffset('MSK', 10800)


Окей, теперь хотим избавиться от тайм-зоны (или привести к какой-нибудь другой зоне, скажем, МСК подойдет)

In [29]:
d = d.astimezone(pytz.timezone('UTC')) 
print(d)
d = d.astimezone(pytz.timezone('Europe/Moscow')) 
print(d)
# Так происходит из-за переходов на летнее-зимнее время (посмотрите про время в Волгограде)

2003-10-11 14:13:46+00:00
2003-10-11 18:13:46+04:00


In [ ]:
#Вывести все возможные таймзоны
for tz in pytz.all_timezones:
    print(tz)

В чем проблема с таймзонами? В том, что с ними в целом сложно (вот у нас был переход на зимнее/летнее время, теперь нет, а потом опять ввели, а потом опять нет)

Поэтому такие косяки бывают всегда, но нам для точности достаточно хотя бы привести все к единому времени (так у нас данные не будут расходиться по часовым поясам)

Хорошо, привели все времена. Хотим теперь, наверное, считать diff между временами? Легко!

In [30]:
dt_1 = dt.datetime(2022, 5, 6, 14, 15, 16)
print(dt_1)
dt_2 = dt.datetime.now() #Дает время в UTC
print(dt_2)

2022-05-06 14:15:16
2022-09-17 12:41:39.008412


In [31]:
dt_2 - dt_1 #Получаем объект типа timedelta

datetime.timedelta(days=133, seconds=80783, microseconds=8412)

In [33]:
diff = dt_2 - dt_1
diff.days #хопа, разница в днях

133

In [34]:
dt_2 + dt.timedelta(days=15) #Хотим узнать, какое число будет через 15 дней

datetime.datetime(2022, 10, 2, 12, 41, 39, 8412)

In [35]:
format = '%d-%m-%Y' # задаем формат через шаблон
dt.datetime.strftime(dt_2 + dt.timedelta(days=15), format) # и красивый вывод по шаблону

'02-10-2022'

А теперь к заданию!

## Задание 6.

Понять, в какие часы и дни играют challengers и выигрывают чаще. Для этого нужно создать поля days и hours с помощью datetime :)

In [38]:
matches.game_datetime = matches.game_datetime.apply(lambda x: parse(x)) # делаем через apply
matches.head(5)

,match_id,puuid,challenger,galaxy,placement,game_datetime,Astro,Chrono,DarkStar,Set3_Celestial,...,Mercenary,Set3_Blademaster,Paragon,Protector,Set3_Sorcerer,StarGuardian,MechPilot,SpacePirate,galaxy_numb,winner
0,NA1_3569517250,7AKHCtexkJJo8p04lg4shtx7n7R-uH6pQwPhBCEofkIJca...,0,Treasure Trove,1,2020-09-10 05:18:02,0,0,3,2,...,0,0,0,1,0,0,0,0,0,1
1,NA1_3569517250,0StQeXmDacsAikvEcY2ubarDZqzlEsaYsZHO6UT-UgBFY7...,0,Treasure Trove,2,2020-09-10 05:18:02,0,0,0,0,...,0,0,1,1,2,2,0,0,0,1
2,NA1_3569517250,Y0emdKI096n9-Rb0L4_I4pP_tmn_kwCtKiEjIsBRrxRqX_...,1,Treasure Trove,3,2020-09-10 05:18:02,0,1,0,0,...,1,0,0,0,0,0,0,0,0,1
3,NA1_3569517250,1vCCOdxLrdYa6NQRhywKKdZ0UNdX0nsa15Zpqo03k6QFs6...,1,Treasure Trove,4,2020-09-10 05:18:02,0,0,1,0,...,0,0,1,1,2,1,0,0,0,1
4,NA1_3569517250,mC0hMfKYhejc1U1aJE7iTOTn6TUDkY4apfQhdw2Pvp_jV3...,0,Treasure Trove,5,2020-09-10 05:18:02,0,0,0,1,...,0,0,1,1,0,0,0,0,0,0


In [ ]:
matches.info() # видим, что datetime

In [39]:
matches['hour'] = matches.game_datetime.apply(lambda x: x.hour)
matches['day'] = matches.game_datetime.apply(lambda x: x.day)

In [42]:
matches[matches.challenger == 1].groupby("hour").agg({"winner": "mean"}).reset_index().sort_values("winner", ascending=False).head(5)
#группируем, берем среднее, смотрим лучшие часы для игр

,hour,winner
13,13,0.800000
22,22,0.781250
11,11,0.777778
9,9,0.774194
0,0,0.750000


In [43]:
matches[matches.challenger == 1].groupby("day").agg({"winner": "mean"}).reset_index().sort_values("winner", ascending=False).head(5)
#группируем, берем среднее, смотрим лучшие дни для игр

,day,winner
5,15,0.750000
1,11,0.709220
3,13,0.705882
6,16,0.666667
4,14,0.653061


## Задание 7.

Сджойнить таблицы с героями и матчами и выбрать самого лучшего чемпиона (самое большое среднее число побед)

In [46]:
merged_matches = pd.merge(matches, heroes, on=['match_id', 'puuid'], how='inner') #делаем merge
merged_matches.sample(5)

,match_id,puuid,challenger,galaxy,placement,game_datetime,Astro,Chrono,DarkStar,Set3_Celestial,...,hour,day,champion,tier,item_set_1,item_set_2,item_set_3,traits_1,traits_2,traits_3
3245,NA1_3569898072,Z7eSk_cM8FNIbdp90YffjQWTXv9heMcglMFkRO3ZHwRRMw...,0,Treasure Trove,8,2020-09-10 16:22:30,0,1,0,0,...,16,10,Riven,1,NaN,NaN,NaN,Chrono,Blademaster,NaN
33019,NA1_3576020423,MG8B13HczMWolE4-FbBWHwV3SVofvUsiwE-9eIs7LBY9_h...,0,Normal Game,3,2020-09-15 16:18:51,0,0,0,1,...,16,15,Xin Zhao,2,NaN,NaN,NaN,Celestial,Protector,NaN
28846,NA1_3575141326,mazQ9lB0gRc6FARQceqjhNU0t2DlT1jCLb7w4JfCSSJV4i...,0,Manatee's Delight,4,2020-09-14 20:14:43,0,0,1,1,...,20,14,Neeko,2,77.0,25.0,69.0,Star Guardian,Protector,NaN
25149,NA1_3574370874,3BpzH6haPgJesmCQmwxibPvWmcYg8VimHvalYykJWPqPvU...,0,Trade Sector,1,2020-09-14 03:30:25,0,1,0,0,...,3,14,Darius,2,99.0,14.0,15.0,Space Pirate,Mana-Reaver,NaN
32174,NA1_3575762465,PfyrAEiKIIO1iz78v778wYrF-lXQEpqZ20_P3caxHfFckO...,0,Manatee's Delight,8,2020-09-15 06:45:59,0,0,0,1,...,6,15,Syndra,2,13.0,44.0,NaN,Star Guardian,Sorcerer,NaN


In [50]:
merged_matches.groupby("champion").agg({"winner": "mean"}).reset_index().sort_values("winner", ascending=False).head(5)

,champion,winner
45,Urgot,0.692008
51,Xerath,0.671458
43,Thresh,0.653930
27,Lulu,0.650273
9,Ekko,0.635000


## Задание 8.

Поменяйте в таблице с героями все Nan на 0, а затем посчитайте и добавьте индекс "полезности" сборки (индекс будем считать как скалярное произведение id итемов на столбец [2, 1, 0] (то есть первый предмет самый нужный, затем второй и только после третий)

Добавьте колонку "coef" в датасет

In [53]:
heroes = heroes.fillna(0) # заменить все Nan на значение (в данном случае 0)
res_1 = np.array(heroes[["item_set_1", "item_set_2", "item_set_3"]]) # Сделали табличку для перемножения
coefs = np.arange(2, -1, -1).reshape(-1, 1) # получили коэффициенты
heroes["coef"] = np.dot(res_1, coefs)
heroes

,match_id,puuid,champion,tier,item_set_1,item_set_2,item_set_3,traits_1,traits_2,traits_3,coef
0,NA1_3576421406,E9vhtL_T6i81fD7EmWshNbUUNuf_gstpkSUI17IcbLW43e...,Caitlyn,1,68.0,88.0,0.0,Chrono,Sniper,0,224.0
1,NA1_3576421406,E9vhtL_T6i81fD7EmWshNbUUNuf_gstpkSUI17IcbLW43e...,Nautilus,2,27.0,59.0,0.0,Astro,Vanguard,0,113.0
2,NA1_3576421406,E9vhtL_T6i81fD7EmWshNbUUNuf_gstpkSUI17IcbLW43e...,Ashe,2,26.0,0.0,0.0,Celestial,Sniper,0,52.0
3,NA1_3576421406,E9vhtL_T6i81fD7EmWshNbUUNuf_gstpkSUI17IcbLW43e...,Karma,2,46.0,0.0,0.0,Dark Star,Mystic,0,92.0
4,NA1_3576421406,E9vhtL_T6i81fD7EmWshNbUUNuf_gstpkSUI17IcbLW43e...,Soraka,2,0.0,0.0,0.0,Star Guardian,Mystic,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
35079,NA1_3569517250,mC0hMfKYhejc1U1aJE7iTOTn6TUDkY4apfQhdw2Pvp_jV3...,Aurelion Sol,2,12.0,37.0,13.0,Rebel,Starship,0,61.0
35080,NA1_3569517250,mC0hMfKYhejc1U1aJE7iTOTn6TUDkY4apfQhdw2Pvp_jV3...,Ekko,1,24.0,0.0,0.0,Cybernetic,Infiltrator,0,48.0
35081,NA1_3569517250,mC0hMfKYhejc1U1aJE7iTOTn6TUDkY4apfQhdw2Pvp_jV3...,Lulu,2,0.0,0.0,0.0,Celestial,Mystic,0,0.0
35082,NA1_3569517250,mC0hMfKYhejc1U1aJE7iTOTn6TUDkY4apfQhdw2Pvp_jV3...,Urgot,2,44.0,69.0,14.0,Battlecast,Protector,0,157.0


In [60]:
# Можно также сделать с помощью apply (да, его можно делать не только для 1 столбца)

def f(x):
    return x[0] * 2 + x[1]

heroes["coef"] = heroes[["item_set_1", "item_set_2", "item_set_3"]].apply(lambda x: f(x), axis=1)
# задаем, что идем по строкам, получаем массив из 3 элементов

## Задание 9.

Посчитать, какие итемы чаще всего используют и как они называются

Иногда для того, чтобы выгрузить данные, вам надо будет подгружать json-файлы. Делается это с помощью библиотеки json

[Документация](https://docs.python.org/3/library/json.html) 

Базово, для того, чтобы выгрузить json, нужно использовать load (когда грузим из файла) или loads (если хотим выгрузить из строки)

In [69]:
import json

with open('items.json') as json_data:
    items = json.load(json_data)
items = pd.DataFrame(items) # А дальше можно просто запихнуть это в DataFrame, очень удобно!

In [ ]:
items

Логичнее, наверное, вначале предрасчитать, а потом это просто добавить к item_id!

In [65]:
res = np.unique(np.array(heroes[["item_set_1", "item_set_2", "item_set_3"]]).ravel(), return_counts=True)
# В unique есть аргумент return_counts, который вам дает еще и подсчет уникальных значений!

count_items = pd.DataFrame(res)
count_items #какая незадача, не то получили, транспонировать бы это

,0,1,2,3,4,5,6,7,8,9,...,45,46,47,48,49,50,51,52,53,54
0,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,66.0,67.0,68.0,69.0,77.0,78.0,79.0,88.0,89.0,99.0
1,65291.0,196.0,351.0,345.0,387.0,272.0,310.0,243.0,49.0,141.0,...,607.0,1464.0,187.0,1381.0,1193.0,328.0,647.0,578.0,221.0,688.0


In [ ]:
count_items = count_items.T #умеем прям как в numpy!
count_items.columns = ["id", "cnt"]
count_items

In [79]:
pd.merge(items, count_items, on=['id'], how='inner').sort_values("cnt", ascending=False).head(5)

,id,name,cnt
13,15,Guardian Angel,2447.0
10,12,Giant Slayer,2041.0
33,44,Seraph's Embrace,1889.0
45,67,Zephyr,1464.0
30,37,Morellonomicon,1403.0


## Задание 10.

Удалите поля со всеми нецелочисленными колонками.

Запишите получившийся результат для матчей в matches_new.csv

In [83]:
matches.select_dtypes(include=['int']) #с помощью select_dtypes можно выбрать только те колонки, которые вам нужны по типу
matches.select_dtypes(include=['int']).to_csv("matches_new.csv")

![Жако](https://petshop-vrn.ru/wp-content/uploads/6/c/a/6ca4dd99334ae553c70fbc4ed4165506.jpeg)

А это Жако, или серый попугай. Считаются самыми умными попугаями в мире, потому что они способны не просто на подражание звуков (как это делают многие попугаи)

В 1980-х годах ученая Ирэн Пепперберг проводила исследования по языковому мышлению различных животных, и одним из подопытных был серый попугай по имени Алекс. По результатам эксперимента Алекс мог произносить очень много слов, причем он верно отвечал на вопросы и в целом коммуницировал исходя не из подражания, а из логики. В целом было показано, что у него было мышление, как у шестилетнего ребенка, что стало полным открытием для ученых.

Хотя, на самом деле, сравнивать мышление попугаев и людей сложно, потому что мышление животных и не должно быть похожим на человеческое